<a href="https://colab.research.google.com/github/JuhyupKim/HY_DIC/blob/main/125-Deeplearning_InitialNdropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Data import

In [ ]:
traindata_url = 'https://bitbucket.org/hyuk125/lg_dic/raw/889649d1bc273bf53967cb621a370205715b803b/dataset_day1/mnist_train.csv'
testdata_url = 'https://bitbucket.org/hyuk125/lg_dic/raw/889649d1bc273bf53967cb621a370205715b803b/dataset_day1/mnist_test.csv'
train_data = pd.read_csv(traindata_url)
test_data = pd.read_csv(testdata_url)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

le.fit(train_data.label == 5)

train_data.label = le.transform(train_data.label == 5)
test_data.label = le.transform(test_data.label == 5)

## Deep learning - classification 모델

### Pytorch 모델에 입력하기 위한 데이터 변환

In [ ]:
train_data = torch.from_numpy(train_data.values).float()
test_data = torch.from_numpy(test_data.values).float()

In [ ]:
BATCH_SIZE = 15
epochs = 2
learning_rate = 0.001
drop_prob = 0.5

In [ ]:

data_loader = torch.utils.data.DataLoader(train_data,
                            batch_size=BATCH_SIZE, 
                            shuffle=True, 
                            num_workers=0)

### Deep learning 모델 정의 with Xavier initializer & dropout

In [ ]:
class DNNModel(nn.Module):
    def __init__(self):
        super(DNNModel, self).__init__()
        self.layer1 = nn.Linear(28 *28, 300)
        self.layer2 = nn.Linear(300, 300)
        self.layer3 = nn.Linear(300, 2)
        self.relu = nn.ReLU()
        
        self.dropout = nn.Dropout(p=drop_prob)
        
        nn.init.xavier_uniform_(self.layer1.weight)
        nn.init.xavier_uniform_(self.layer2.weight)
        nn.init.xavier_uniform_(self.layer3.weight)
    
        
    def forward(self, x):
        
        layers = nn.Sequential(self.layer1, self.relu, self.dropout,
                           self.layer2, self.relu, self.dropout,
                           self.layer3).to(device)
        out = layers(x)
        return out
    
model = DNNModel()
model

### 학습 시작

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
for epoch in range(epochs):
    running_cost = 0.0

    for step, (batch_data) in enumerate(data_loader):
        batch_x = batch_data[:, 1:].view(-1, 28*28).to(device)
        batch_y = batch_data[:, 0].to(device).long()
        
        optimizer.zero_grad()
        
        outputs = model(batch_x)
        cost = criterion(outputs, batch_y)

        cost.backward()
        optimizer.step()
        
        running_cost += cost.item()
        if step % 200 == 199:
            print('[%d, %5d] cost: %.3f' % (epoch + 1, step + 1, running_cost / 200))
            running_cost = 0.0
            

## 정확도 판단

### Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

In [ ]:
with torch.no_grad():
    X_test = test_data[:, 1:].view(-1, 28 * 28).float().to(device)
    y_test = test_data[:, 0].float()
    
    prediction = model(X_test).cpu()
    print(confusion_matrix(torch.argmax(prediction, 1), y_test))
    print("==Precision==")
    print(precision_score(torch.argmax(prediction, 1), y_test, average=None))
    print(precision_score(torch.argmax(prediction, 1), y_test, average='weighted'))
    print("Recall")
    print(recall_score(torch.argmax(prediction, 1), y_test, average=None))
    print(recall_score(torch.argmax(prediction, 1), y_test, average='weighted'))